### All the necessary Imports

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Defining League and Seasons to consider

In [ ]:
league_key = {'England':1,'Spain':31,'Germany':32,'Italy':33,'France':34,'Holland':36,'Portugal':37,'Scotland':10,'Turkey':79,'Championship':2}

In [ ]:
season_key = {2019:16,2018:15,2017:14,2016:13,2015:12,2014:11,2013:10,2012:9,2011:8,2010:7,2009:6,2008:5,2007:4,2006:3}

### Defining a function to split one list into a list of smaller lists

In [ ]:
def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n]

### Defining a function to get league data based on input League and Season

In [ ]:
def get_league_data(league,season):
    #Building the URL
    data_url = 'https://thefishy.co.uk/leaguetable.php?table='+str(league_key[league])+'&season='+str(season_key[season])
    
    #Use Beautiful Soup to pull all contents of the web-page
    page = requests.get(data_url)
    soup = BeautifulSoup(page.content,'html.parser')
    
    #Find all tables in the page
    tables = soup.findAll("table")
    
    #Selecting the correct table to pull data from
    table_to_use = tables[5]
    
    #Extracting the header details
    table_header = table_to_use.find_all('th')
    header_list = []
    for i in table_header:
        header_list.append(i.contents[0])
        
    #Extracting table contents and cleaning data
    table_data = table_to_use.find_all('td')
    
    #Cleaning the table by removing tags
    table_data_clean = []
    for i in table_data:
        table_data_clean.append(i.contents[0])
            
    #Converting the larger list into a list of lists
    n = 10  
    x = list(divide_chunks(table_data_clean, n))
    
    #Converting the list of lists into a dataframe
    df = pd.DataFrame(x, columns =header_list)
    
    #Removing the tag values from the Team column
    df_team = df['Team']
    for i in range(0,len(df_team)):
        df_team[i] = df_team[i].contents[0]
    df['Team'] = df_team
    return df

In [10]:
get_league_data('Turkey',2019).to_csv('Turkish_Super_Lig_2019.csv',index=False)

In [9]:
get_league_data('Germany',2007).to_csv('Bundesliga_2007.csv',index=False)
get_league_data('Germany',2008).to_csv('Bundesliga_2008.csv',index=False)
get_league_data('Germany',2009).to_csv('Bundesliga_2009.csv',index=False)
get_league_data('Germany',2010).to_csv('Bundesliga_2010.csv',index=False)
get_league_data('Germany',2011).to_csv('Bundesliga_2011.csv',index=False)
get_league_data('Germany',2012).to_csv('Bundesliga_2012.csv',index=False)
get_league_data('Germany',2013).to_csv('Bundesliga_2013.csv',index=False)
get_league_data('Germany',2014).to_csv('Bundesliga_2014.csv',index=False)
get_league_data('Germany',2015).to_csv('Bundesliga_2015.csv',index=False)
get_league_data('Germany',2016).to_csv('Bundesliga_2016.csv',index=False)
get_league_data('Germany',2017).to_csv('Bundesliga_2017.csv',index=False)
get_league_data('Germany',2018).to_csv('Bundesliga_2018.csv',index=False)
get_league_data('Germany',2019).to_csv('Bundesliga_2019.csv',index=False)
get_league_data('Championship',2007).to_csv('English_Championship_2007.csv',index=False)
get_league_data('Championship',2008).to_csv('English_Championship_2008.csv',index=False)
get_league_data('Championship',2009).to_csv('English_Championship_2009.csv',index=False)
get_league_data('Championship',2010).to_csv('English_Championship_2010.csv',index=False)
get_league_data('Championship',2011).to_csv('English_Championship_2011.csv',index=False)
get_league_data('Championship',2012).to_csv('English_Championship_2012.csv',index=False)
get_league_data('Championship',2013).to_csv('English_Championship_2013.csv',index=False)
get_league_data('Championship',2014).to_csv('English_Championship_2014.csv',index=False)
get_league_data('Championship',2015).to_csv('English_Championship_2015.csv',index=False)
get_league_data('Championship',2016).to_csv('English_Championship_2016.csv',index=False)
get_league_data('Championship',2017).to_csv('English_Championship_2017.csv',index=False)
get_league_data('Championship',2018).to_csv('English_Championship_2018.csv',index=False)
get_league_data('Championship',2019).to_csv('English_Championship_2019.csv',index=False)
get_league_data('England',2007).to_csv('English_Premiership_2007.csv',index=False)
get_league_data('England',2008).to_csv('English_Premiership_2008.csv',index=False)
get_league_data('England',2009).to_csv('English_Premiership_2009.csv',index=False)
get_league_data('England',2010).to_csv('English_Premiership_2010.csv',index=False)
get_league_data('England',2011).to_csv('English_Premiership_2011.csv',index=False)
get_league_data('England',2012).to_csv('English_Premiership_2012.csv',index=False)
get_league_data('England',2013).to_csv('English_Premiership_2013.csv',index=False)
get_league_data('England',2014).to_csv('English_Premiership_2014.csv',index=False)
get_league_data('England',2015).to_csv('English_Premiership_2015.csv',index=False)
get_league_data('England',2016).to_csv('English_Premiership_2016.csv',index=False)
get_league_data('England',2017).to_csv('English_Premiership_2017.csv',index=False)
get_league_data('England',2018).to_csv('English_Premiership_2018.csv',index=False)
get_league_data('England',2019).to_csv('English_Premiership_2019.csv',index=False)
get_league_data('Holland',2007).to_csv('Eredivise_2007.csv',index=False)
get_league_data('Holland',2008).to_csv('Eredivise_2008.csv',index=False)
get_league_data('Holland',2009).to_csv('Eredivise_2009.csv',index=False)
get_league_data('Holland',2010).to_csv('Eredivise_2010.csv',index=False)
get_league_data('Holland',2011).to_csv('Eredivise_2011.csv',index=False)
get_league_data('Holland',2012).to_csv('Eredivise_2012.csv',index=False)
get_league_data('Holland',2013).to_csv('Eredivise_2013.csv',index=False)
get_league_data('Holland',2014).to_csv('Eredivise_2014.csv',index=False)
get_league_data('Holland',2015).to_csv('Eredivise_2015.csv',index=False)
get_league_data('Holland',2016).to_csv('Eredivise_2016.csv',index=False)
get_league_data('Holland',2017).to_csv('Eredivise_2017.csv',index=False)
get_league_data('Holland',2018).to_csv('Eredivise_2018.csv',index=False)
get_league_data('Holland',2019).to_csv('Eredivise_2019.csv',index=False)
get_league_data('Spain',2007).to_csv('La_Liga_2007.csv',index=False)
get_league_data('Spain',2008).to_csv('La_Liga_2008.csv',index=False)
get_league_data('Spain',2009).to_csv('La_Liga_2009.csv',index=False)
get_league_data('Spain',2010).to_csv('La_Liga_2010.csv',index=False)
get_league_data('Spain',2011).to_csv('La_Liga_2011.csv',index=False)
get_league_data('Spain',2012).to_csv('La_Liga_2012.csv',index=False)
get_league_data('Spain',2013).to_csv('La_Liga_2013.csv',index=False)
get_league_data('Spain',2014).to_csv('La_Liga_2014.csv',index=False)
get_league_data('Spain',2015).to_csv('La_Liga_2015.csv',index=False)
get_league_data('Spain',2016).to_csv('La_Liga_2016.csv',index=False)
get_league_data('Spain',2017).to_csv('La_Liga_2017.csv',index=False)
get_league_data('Spain',2018).to_csv('La_Liga_2018.csv',index=False)
get_league_data('Spain',2019).to_csv('La_Liga_2019.csv',index=False)
get_league_data('France',2007).to_csv('Ligue1_2007.csv',index=False)
get_league_data('France',2008).to_csv('Ligue1_2008.csv',index=False)
get_league_data('France',2009).to_csv('Ligue1_2009.csv',index=False)
get_league_data('France',2010).to_csv('Ligue1_2010.csv',index=False)
get_league_data('France',2011).to_csv('Ligue1_2011.csv',index=False)
get_league_data('France',2012).to_csv('Ligue1_2012.csv',index=False)
get_league_data('France',2013).to_csv('Ligue1_2013.csv',index=False)
get_league_data('France',2014).to_csv('Ligue1_2014.csv',index=False)
get_league_data('France',2015).to_csv('Ligue1_2015.csv',index=False)
get_league_data('France',2016).to_csv('Ligue1_2016.csv',index=False)
get_league_data('France',2017).to_csv('Ligue1_2017.csv',index=False)
get_league_data('France',2018).to_csv('Ligue1_2018.csv',index=False)
get_league_data('France',2019).to_csv('Ligue1_2019.csv',index=False)
get_league_data('Portugal',2007).to_csv('Premiera_Liga_2007.csv',index=False)
get_league_data('Portugal',2008).to_csv('Premiera_Liga_2008.csv',index=False)
get_league_data('Portugal',2009).to_csv('Premiera_Liga_2009.csv',index=False)
get_league_data('Portugal',2010).to_csv('Premiera_Liga_2010.csv',index=False)
get_league_data('Portugal',2011).to_csv('Premiera_Liga_2011.csv',index=False)
get_league_data('Portugal',2012).to_csv('Premiera_Liga_2012.csv',index=False)
get_league_data('Portugal',2013).to_csv('Premiera_Liga_2013.csv',index=False)
get_league_data('Portugal',2014).to_csv('Premiera_Liga_2014.csv',index=False)
get_league_data('Portugal',2015).to_csv('Premiera_Liga_2015.csv',index=False)
get_league_data('Portugal',2016).to_csv('Premiera_Liga_2016.csv',index=False)
get_league_data('Portugal',2017).to_csv('Premiera_Liga_2017.csv',index=False)
get_league_data('Portugal',2018).to_csv('Premiera_Liga_2018.csv',index=False)
get_league_data('Portugal',2019).to_csv('Premiera_Liga_2019.csv',index=False)
get_league_data('Scotland',2007).to_csv('Scottish_Premiership_2007.csv',index=False)
get_league_data('Scotland',2008).to_csv('Scottish_Premiership_2008.csv',index=False)
get_league_data('Scotland',2009).to_csv('Scottish_Premiership_2009.csv',index=False)
get_league_data('Scotland',2010).to_csv('Scottish_Premiership_2010.csv',index=False)
get_league_data('Scotland',2011).to_csv('Scottish_Premiership_2011.csv',index=False)
get_league_data('Scotland',2012).to_csv('Scottish_Premiership_2012.csv',index=False)
get_league_data('Scotland',2013).to_csv('Scottish_Premiership_2013.csv',index=False)
get_league_data('Scotland',2014).to_csv('Scottish_Premiership_2014.csv',index=False)
get_league_data('Scotland',2015).to_csv('Scottish_Premiership_2015.csv',index=False)
get_league_data('Scotland',2016).to_csv('Scottish_Premiership_2016.csv',index=False)
get_league_data('Scotland',2017).to_csv('Scottish_Premiership_2017.csv',index=False)
get_league_data('Scotland',2018).to_csv('Scottish_Premiership_2018.csv',index=False)
get_league_data('Scotland',2019).to_csv('Scottish_Premiership_2019.csv',index=False)
get_league_data('Italy',2007).to_csv('Serie_A_2007.csv',index=False)
get_league_data('Italy',2008).to_csv('Serie_A_2008.csv',index=False)
get_league_data('Italy',2009).to_csv('Serie_A_2009.csv',index=False)
get_league_data('Italy',2010).to_csv('Serie_A_2010.csv',index=False)
get_league_data('Italy',2011).to_csv('Serie_A_2011.csv',index=False)
get_league_data('Italy',2012).to_csv('Serie_A_2012.csv',index=False)
get_league_data('Italy',2013).to_csv('Serie_A_2013.csv',index=False)
get_league_data('Italy',2014).to_csv('Serie_A_2014.csv',index=False)
get_league_data('Italy',2015).to_csv('Serie_A_2015.csv',index=False)
get_league_data('Italy',2016).to_csv('Serie_A_2016.csv',index=False)
get_league_data('Italy',2017).to_csv('Serie_A_2017.csv',index=False)
get_league_data('Italy',2018).to_csv('Serie_A_2018.csv',index=False)
get_league_data('Italy',2019).to_csv('Serie_A_2019.csv',index=False)